In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing

%matplotlib inline

2024-01-26 13:29:43.985273: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-26 13:29:44.008240: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-26 13:29:44.008265: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-26 13:29:44.008282: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-26 13:29:44.012676: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-26 13:29:44.012963: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
attacks = pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/csvDataFeaturesTest.csv", sep=";")

In [3]:
attacks = preprocessing.normalize(attacks.to_numpy(), norm="max", axis=0)

# should probably add batch and shuffle

train_dataset = attacks[:int(np.floor(attacks.shape[0]*3/4))]
test_dataset = attacks[int(np.floor(attacks.shape[0]*3/4)):]

num_features = attacks[:int(np.floor(attacks.shape[0]*3/4))].shape[1]
seq_length = 5

In [4]:
def wasserstein_loss(y_true, y_pred):
    return tf.keras.backend.mean(y_true * y_pred)

In [5]:
# clip model weights to a given hypercube
class ClipConstraint(tf.keras.constraints.Constraint):
	# set clip value when initialized
	def __init__(self, clip_value):
		self.clip_value = clip_value

	# clip model weights to hypercube
	def __call__(self, weights):
		return tf.keras.backend.clip(weights, -self.clip_value, self.clip_value)

	# get the config
	def get_config(self):
		return {'clip_value': self.clip_value}

In [73]:
# noise will have same dimension as one flow

def make_generator_model(input_dim):
    input = tf.keras.layers.Input(shape=input_dim, name="generator input")
    hidden = tf.keras.layers.Dense(80, activation="relu")(input)
    hidden = tf.keras.layers.Dense(80, activation="relu")(hidden)

    sport_hidden = tf.keras.layers.Dense(1, name="sport_hidden")(hidden)
    dport_hidden = tf.keras.layers.Dense(1, name="dport_hidden")(hidden)
    proto_hidden = tf.keras.layers.Dense(1, name="proto_hidden")(hidden)

    # dense output for continuous, softmax for categorical
    sport_output = tf.keras.layers.Softmax(1, name="sport_output")(sport_hidden)
    dport_output = tf.keras.layers.Softmax(1, name="dport_output")(dport_hidden)
    proto_output = tf.keras.layers.Softmax(1, name="proto_output")(proto_hidden)
    cont_output = tf.keras.layers.Dense(num_features-3, name="cont_output")(hidden)
    output = tf.keras.layers.concatenate([sport_output, dport_output, proto_output, cont_output])
    model = tf.keras.Model(inputs=input, outputs=output, name="Generator")
    # model = tf.keras.Model(inputs=input, outputs=[sport_output, dport_output, proto_output, cont_output])
    return model

In [74]:
generator = make_generator_model(50)
t = generate_latent_points(50, 2)
# generator.predict(t)[0].shape, generator.predict(t)[1].shape, generator.predict(t)[2].shape, generator.predict(t)[3].shape
generator.predict(t).shape

1/1 [==============================] - 0s 50ms/step


(2, 22)

In [37]:
def make_critic_model(input_shape):
    const = ClipConstraint(0.01)

    # model = tf.keras.Sequential()
    # model.add(tf.keras.layers.LSTM(100, input_shape=input_shape, recurrent_dropout=0.4, return_sequences=True))
    # model.add(tf.keras.layers.BatchNormalization())
    # model.add(tf.keras.layers.LSTM(100, recurrent_dropout=0.4))
    # model.add(tf.keras.layers.BatchNormalization())
    # # model.add(tf.keras.layers.Dropout(0.2))
    # # what should the units of this dense layer be?
    # # seems like it outputs one value >0 for real data and <0 for fake data
    # # for each item in the batch
    # # linear activation since it's a critic
    # model.add(tf.keras.layers.Dense(1, activation="linear"))

    input = tf.keras.layers.Input(shape=input_shape, name="discriminator input")
    hidden = tf.keras.layers.LSTM(100, recurrent_dropout=0.4, return_sequences=True, kernel_constraint=const)(input)
    hidden = tf.keras.layers.BatchNormalization()(hidden)
    hidden = tf.keras.layers.LSTM(100, recurrent_dropout=0.4, kernel_constraint=const)(hidden)
    hidden = tf.keras.layers.BatchNormalization()(hidden)
    # linear activation for output layer since it's a critic
    output = tf.keras.layers.Dense(1, activation="linear")(hidden)

    model = tf.keras.Model(inputs=input, outputs=output, name="Critic")

    opt = tf.keras.optimizers.RMSprop(learning_rate = 0.00005)
    model.compile(loss=wasserstein_loss, optimizer=opt)
    return model

In [36]:
# def make_gan(generator, critic):
#     # make weights in critic not trainable
#     for layer in critic.layers:
#         if not isinstance(layer, tf.keras.layers.BatchNormalization):
#             layer.trainable = False
#     model = tf.keras.Sequential()
#     model.add(generator)
#     model.add(critic)
#     opt = tf.keras.optimizers.RMSprop(lr=0.00005)
#     model.compile(loss=wasserstein_loss, optimizer=opt)
#     return model

def make_gan(generator, critic, gen_input_dim):
    for layer in critic.layers:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False

    input = tf.keras.layers.Input(shape=gen_input_dim, name="combined input")
    x = generator(input)
    x = critic(x)
    model = tf.keras.Model(inputs=input, outputs=x)
    opt = tf.keras.optimizers.RMSprop(learning_rate=0.00005)
    model.compile(loss=wasserstein_loss, optimizer=opt)
    return model

In [15]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = np.random.randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [14]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = np.random.randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels, -1 for 'real'
	y = -np.ones((n_samples, 1))
	return X, y

In [11]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels with 1.0 for 'fake'
	y = np.ones((n_samples, 1))
	return X, y

In [79]:
# train the generator and critic
def train(g_model, c_model, gan_model, dataset, latent_dim, n_epochs=2, n_batch=64, n_critic=5):
	# calculate the number of batches per training epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# calculate the size of half a batch of samples
	half_batch = int(n_batch / 2)
	# lists for keeping track of loss
	c1_hist, c2_hist, g_hist = list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# update the critic more than the generator
		c1_tmp, c2_tmp = list(), list()
		for _ in range(n_critic):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update critic model weights
			c_loss1 = c_model.train_on_batch(X_real, y_real)
			c1_tmp.append(c_loss1)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update critic model weights
			c_loss2 = c_model.train_on_batch(X_fake, y_fake)
			c2_tmp.append(c_loss2)
		# store critic loss
		c1_hist.append(np.mean(c1_tmp))
		c2_hist.append(np.mean(c2_tmp))
		# prepare points in latent space as input for the generator
		X_gan = generate_latent_points(latent_dim, n_batch)
		print(X_gan.shape)
		# create inverted labels for the fake samples
		y_gan = -np.ones((n_batch, 1))
		# update the generator via the critic's error
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		g_hist.append(g_loss)
		# summarize loss on this batch
		print('>%d, c1=%.3f, c2=%.3f g=%.3f' % (i+1, c1_hist[-1], c2_hist[-1], g_loss))
		plot_history(c1_hist, c2_hist, g_hist)


In [80]:
# size of the latent space
latent_dim = 50
# create the critic
critic = make_critic_model((num_features, 1))
# create the generator
generator = make_generator_model(latent_dim)
# create the gan
gan_model = make_gan(generator, critic, latent_dim)
# load image data
dataset = attacks
print(dataset.shape)
# train model
train(generator, critic, gan_model, dataset, latent_dim)

(248278, 22)
1/1 [==============================] - 0s 12ms/step
(64, 50)
>1, c1=-0.001, c2=0.002 g=-0.001
1/1 [==============================] - 0s 9ms/step
(64, 50)
>2, c1=-0.003, c2=0.003 g=-0.003
1/1 [==============================] - 0s 10ms/step
(64, 50)
>3, c1=-0.004, c2=0.004 g=-0.004
1/1 [==============================] - 0s 9ms/step
(64, 50)
>4, c1=-0.005, c2=0.006 g=-0.005
1/1 [==============================] - 0s 9ms/step
(64, 50)
>5, c1=-0.006, c2=0.006 g=-0.006
1/1 [==============================] - 0s 9ms/step
(64, 50)
>6, c1=-0.007, c2=0.007 g=-0.007
1/1 [==============================] - 0s 9ms/step
(64, 50)
>7, c1=-0.008, c2=0.008 g=-0.008
1/1 [==============================] - 0s 9ms/step
(64, 50)
>8, c1=-0.008, c2=0.009 g=-0.008
1/1 [==============================] - 0s 9ms/step
(64, 50)
>9, c1=-0.009, c2=0.010 g=-0.009
1/1 [==============================] - 0s 9ms/step
(64, 50)
>10, c1=-0.010, c2=0.010 g=-0.010
1/1 [==============================] - 0s 9ms/step
(64

In [78]:
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist):
 # plot history
 plt.plot(d2_hist, label='crit_fake')
 plt.plot(g_hist, label='gen')
 plt.legend()
 plt.savefig('plot_line_plot_loss.png')
 plt.close()